In the context of the **Integrated Reporting Framework (IReF)** project, I collected data of MALTA that needed to be analyzed and preprocessed.  
After having collected some statistics, I generated the outpul file ready to be uploaded to our system. 

IReF is an ECB project, its aim is to integrate the Eurosystem’s statistical requirements for banks into a single standardised reporting framework that would be applicable across the euro area and that might also be adopted by authorities in other EU Member States.

More details can be found here: https://www.ecb.europa.eu/stats/ecb_statistics/co-operation_and_standards/reporting/html/index.en.html

In [2]:
import pandas as pd
import pyodbc
import os
import datetime

In [3]:
cnxn = pyodbc.connect('DSN=DISC DP Impala 64bit',autocommit=True)
cursor = cnxn.cursor()

In [8]:
query_cd = '''
query here
'''

In [9]:
df =  pd.read_sql_query(query_cd, cnxn)
df['primary_code'] = 'Not Publicly Available'
df['entty_cd'] = 'Not Publicly Available'
df.sample(5)

C:\Users\bressan\AppData\Local\Temp\ipykernel_22104\2856501524.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df =  pd.read_sql_query(query_cd, cnxn)


,primary_code,vld_frm,vld_t,cty,cntry,entty_cd,cnf_code,src,esa sector
27846,Not Publicly Available,2021-07-31,9999-12-31,SAN GILJAN,MT,Not Publicly Available,N,STAT,S126
148886,Not Publicly Available,2017-11-20,2017-11-30,MOSTA,MT,Not Publicly Available,F,STAT,S125
8487,Not Publicly Available,2019-01-31,2019-08-30,ATTARD,MT,Not Publicly Available,F,STAT,S11
109014,Not Publicly Available,2018-03-01,9999-12-31,MRIEHEL INDUSTRIAL ZONE,MT,Not Publicly Available,F,STAT,S11
105907,Not Publicly Available,2019-09-30,2019-12-31,IL-FURJANA,MT,Not Publicly Available,N,STAT,S11


In [96]:
MALTA1 = pd.DataFrame(columns= ['ID','CODE', 'VALIFITYF','VALIDITYT','SOURCE','CONF', 'VALUE', 'REQUEST'])

In [ ]:
#change the confidentiality to FREE for MT values
MALTA_TOT= pd.DataFrame({'ID': ['RIAD_TYP_ENTTY_CD_C' for i in range(len(df))],
                              'CODE': list(df['entty_riad_cd']),
                              'VALIFITYF': list(df['vld_frm']), 
                              'VALIFITYT': list(df['vld_t']),
                              'SOURCE': list(df['src']),
                              'CONF': ['F' for i in range(len(df))],
                              'VALUE': list(df['entty_cd']),
                              'REQUEST': ['UP' for i in range(len(df))]
                             })

In [121]:
output_path_upload = 'C:/Users/USER/Desktop/MALTA_ISC.xlsx'
P= pd.DataFrame({'1' : ['Survey','Sender','MessageID','timeProduction'], '2':['FILEP','MT', '10001', '2017-01-18T12:00:00']})
with pd.ExcelWriter(output_path_upload) as writer:
    P.to_excel(writer, sheet_name='FILEP', header= None, index=None)
    MALTA_TOT.to_excel(writer, sheet_name='FILEP', index=None, startrow=4)

In [18]:
fromBO = pd.read_csv('DETAILS.csv')
fromBO.sample(5)

,ENTTY_RIAD_CD,OBS_VALUE,INSTTTNL_SCTR_DTL
1627,Not Publicly Available,S126,S126_A
2110,Not Publicly Available,S11,-4
2852,Not Publicly Available,S127,S127_A
4801,Not Publicly Available,S11,-4
7552,Not Publicly Available,S11,-4


In [ ]:
merged = pd.merge(fromBO, MALTA_TOT, on=['primary_code'], how='inner')
merged

In [149]:
#output to be uploaded to the db
merged.to_csv('MT_merged.csv')